In [1]:
!pip install datasets


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from datasets import load_from_disk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from collections import Counter
import re
import json


In [3]:
# pandas datasframe으로 변환 및 필요한 컬럼만 추출하는 함수
def dataset_to_df(dataset):
  data_list = []

  for data in dataset:
        data_list.append({
            'title': data["title"],
            "context": data["context"],
            "question": data["question"],
            "id": data["id"],
            "answers": data["answers"],
            "document_id": data["document_id"],
            "__index_level_0__": data["__index_level_0__"]
          })  
  df = pd.DataFrame(data_list)
  return df
def dataset_to_dftest(dataset):
  data_list = []

  for data in dataset:
    data_list.append({
        'question' : data["question"],
        "id" : data["id"],
   
    })
  df = pd.DataFrame(data_list)
  return df




In [4]:
import re
from collections import Counter

def count_markdown_elements(data_samples):
    total_counts = Counter()
    
    for data in data_samples:
        markdown_elements = {
            'headers': len(re.findall(r'^#{1,6}\s', data, re.MULTILINE)),
            'bold': len(re.findall(r'\*\*.*?\*\*', data)),
            'italic': len(re.findall(r'\*.*?\*', data)),
            'code_blocks': len(re.findall(r'```[\s\S]*?```', data)),
            'inline_code': len(re.findall(r'`[^`\n]+`', data)),
            'links': len(re.findall(r'\[.*?\]\(.*?\)', data)),
            'lists': len(re.findall(r'^\s*[-*+]\s+\S', data, re.MULTILINE))
        }
        total_counts.update(markdown_elements)
    
    return dict(total_counts)

# 사용 예시
# markdown_counts = count_markdown_elements(df['text'])
# print(markdown_counts)

In [5]:
def remove_markdown(text): #마크 다운 삭제
    # 헤더 제거
    text = re.sub(r'^#{1,6}\s', '', text, flags=re.MULTILINE)
    # 볼드체 제거
    text = re.sub(r'\*\*(.*?)\*\*', r'\1', text)
    # 이탤릭체 제거
    text = re.sub(r'\*(.*?)\*', r'\1', text)
    # 코드 블록 제거
    text = re.sub(r'```[\s\S]*?```', '', text)
    # 인라인 코드 제거
    text = re.sub(r'`([^`\n]+)`', r'\1', text)
    # 링크 제거 (텍스트만 유지)
    text = re.sub(r'\[(.*?)\]\(.*?\)', r'\1', text)
    # 리스트 기호 제거
    text = re.sub(r'^\s*[-*+]\s', '', text, flags=re.MULTILINE)
    return text.strip()

In [6]:
# 개행 문자 카운트 함수 (정휘님  \\n 뿐만 아니라 \n 도 있었던거 같아 확인하기)
def get_newline(data_samples):
    all_lines = []
    for i,data in enumerate(data_samples):
        new_line_count = data.count("\\n")
        new1_line_count = data.count("\n")
        all_lines.append([i, new_line_count,new1_line_count])
    return all_lines

In [7]:
path = "../data/train_dataset/"
dataset = load_from_disk(path)

In [8]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})


In [9]:
def load_wikipedia_data(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        wiki = json.load(f)

In [10]:
# wikipedia 데이터 불러오기
with open(".json", "r", encoding="utf-8") as f:
    wiki = json.load(f)

wiki = pd.DataFrame(wiki)
wiki = wiki.transpose() # 필요한 column만 남기기
wiki.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [11]:
train_df = dataset_to_df(dataset["train"])

In [12]:
train_df.head()

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548


In [13]:
validation = dataset_to_df(dataset["validation"])

In [14]:
test = dataset_to_dftest(dataset["validation"])

In [15]:
print(len(train_df))
print(len(validation))
print(len(test))
print(len(wiki))

3952
240
240
60613


In [65]:
# train_df
# validation
# test
# wiki

In [66]:
#함수 모음

In [16]:
train_df = dataset_to_df(dataset["train"])

In [17]:
# context와 newline 를 구별하기 위해 newline에 대한 \\n제거 확인
train_df["context"] = train_df["context"].str.replace("\\n", " ", regex=False)

In [18]:
train_df["context"] = train_df["context"].str.replace(r'\s+', ' ', regex=True)

In [19]:
train_df_newline = train_df.copy() #train_df_newline에 저장

In [20]:
newline_counts = np.array(get_newline(train_df_newline["context"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))

\\n count: 0
\n count: 0


In [21]:
train_df = dataset_to_df(dataset["train"]) # 초기화 

In [22]:
train_df['context'] = train_df['context'].apply(remove_markdown)  # 마크다운 진행

In [23]:
train_df_mkdown = train_df.copy() # 마크다운 파일 저장

In [24]:
print(count_markdown_elements(train_df_mkdown['context']))

{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [25]:
train_df = dataset_to_df(dataset["train"]) # 초기화 

In [26]:
train_df["context"] = train_df["context"].str.replace("\\n", " ", regex=False) # \n 제거 
train_df["context"] = train_df["context"].str.replace(r'\s+', ' ', regex=True) # 공백 제거 
train_df['context'] = train_df['context'].apply(remove_markdown)  # 마크다운 진행
newline_counts = np.array(get_newline(train_df["context"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))
print(count_markdown_elements(train_df['context']))
# train_df 2가지 모두 적용버전

\\n count: 0
\n count: 0
{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [359]:
#사용할 함수들
#train_df
#train_df_newline
#train_df_mkdown

In [22]:
#mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm

In [27]:
#wiki = pd.read_csv("../wiki_unique.csv") wiki 초기화 
wiki_df = load_wikipedia_data(".json")

In [28]:
newline_counts = np.array(get_newline(wiki["text"]))

In [29]:
# 개행 문자 제거
wiki['text'] = wiki['text'].replace(['\n', '\\\\n'], ' ', regex=True)

In [30]:
wiki["text"] = wiki["text"].str.replace(r'\s+', ' ', regex=True) # 중복 띄어쓰기 제거 

In [31]:
wiki_newline = wiki.copy() # 개행 저장
newline_counts = np.array(get_newline(wiki_newline["text"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))

\\n count: 0
\n count: 0


In [32]:
wiki_df = load_wikipedia_data(".json")

In [33]:
# wiki DataFrame의 'text' 열에 함수 적용
wiki['text'] = wiki['text'].apply(remove_markdown)

In [34]:
wiki_mkdown = wiki.copy()
print(count_markdown_elements(wiki_mkdown['text']))

{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [35]:
wiki_df = load_wikipedia_data(".json")# 초기화

In [36]:
wiki['text'] = wiki['text'].replace(['\n', '\\\\n'], ' ', regex=True) # 개행 문자 제거
wiki["text"] = wiki["text"].str.replace(r'\s+', ' ', regex=True) # 중복 띄어쓰기 제거 
wiki['text'] = wiki['text'].apply(remove_markdown)
newline_counts = np.array(get_newline(wiki["text"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))
print(count_markdown_elements(wiki['text']))

\\n count: 0
\n count: 0
{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [145]:
#사용할 함수들
#wiki_neline 
#wiki_mkdown
#wiki

In [146]:
#mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm

In [37]:
validation = dataset_to_df(dataset["validation"])

In [38]:
newline_counts = np.array(get_newline(validation["context"]))

In [39]:
validation['context'] = validation['context'].replace(['\n', '\\\\n'], ' ', regex=True) # 개행 문자 제거

In [40]:
validation["context"] = validation["context"].str.replace(r'\s+', ' ', regex=True) # 띄어쓰기 제거 

In [41]:
validation_newline =validation.copy()
# 개행 문자 카운트
newline_counts = np.array(get_newline(validation_newline["context"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))

\\n count: 0
\n count: 0


In [42]:
validation = dataset_to_df(dataset["validation"])

In [43]:
validation['context'] = validation['context'].apply(remove_markdown) # 마크다운 제거 

In [44]:
validation_mkdown =validation.copy()
print(count_markdown_elements(validation_mkdown['context']))

{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [45]:
validation = dataset_to_df(dataset["validation"]) # 초기화 

In [46]:
validation['context'] = validation['context'].replace(['\n', '\\\\n'], ' ', regex=True) # 개행 문자 제거
validation["context"] = validation["context"].str.replace(r'\s+', ' ', regex=True) # 띄어쓰기 제거 
validation['context'] = validation['context'].apply(remove_markdown) # 마크다운 제거 
newline_counts = np.array(get_newline(validation["context"]))
print("\\\\n count:", np.sum(newline_counts[:,1], axis=0))
print("\\n count:", np.sum(newline_counts[:,2], axis=0))
print(count_markdown_elements(validation_mkdown['context']))

\\n count: 0
\n count: 0
{'headers': 0, 'bold': 0, 'italic': 0, 'code_blocks': 0, 'inline_code': 0, 'links': 0, 'lists': 0}


In [284]:
# train_df
# validation
# test
# wiki

# validation_newline 
# validation_mkdown
# validation

#train_df
#train_df_newline
#train_df_mkdown


#wiki_newline 
#wiki_mkdown
#wiki

In [ ]:

##-------------------------------------------------------------------------------------

In [49]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# 데이터셋 목록
#datasets = ['validation', 'train_df']
datasets = ['wiki']
versions = ['', '_newline', '_mkdown']

# 각 데이터셋에 대해 반복
for dataset in datasets:
    for version in versions:
        # 데이터프레임 이름 (예: validation_newline)
        df_name = f"{dataset}{version}"
        
        # 데이터프레임이 존재하는지 확인
        if df_name in globals():
            df = globals()[df_name]
            
            # CSV 파일로 저장
            csv_filename = f"{df_name}.csv"
            df.to_csv(csv_filename, index=False)
            print(f"Saved {csv_filename}")
                      

print("All files have been saved.")

Saved wiki.csv
Saved wiki_newline.csv
Saved wiki_mkdown.csv
All files have been saved.


In [86]:
from datasets import Dataset
train_df =  Dataset.from_pandas(train_df)
train_df_newline = Dataset.from_pandas(train_df_newline)
train_df_mkdown = Dataset.from_pandas(train_df_mkdown)

validation = Dataset.from_pandas(validation)
validation_newline = Dataset.from_pandas(validation_newline)
validation_mkdown = Dataset.from_pandas(validation_mkdown)

In [ ]:
# 데이터셋 목록
datasets = ['validation', 'train_df', 'wiki']
versions = ['', '_newline', '_mkdown']

# 각 데이터셋에 대해 반복
for dataset in datasets:
    for version in versions:
        # 데이터셋 이름 (예: validation_newline)
        dataset_name = f"{dataset}{version}"
        
        # 데이터셋이 존재하는지 확인
        if dataset_name in globals():
            dataset_obj = globals()[dataset_name]
            
            # 데이터셋이 이미 Dataset 객체인지 확인
            if not isinstance(dataset_obj, Dataset):
                # pandas DataFrame을 Dataset으로 변환
                dataset_obj = Dataset.from_pandas(dataset_obj)
            
            # Arrow 파일로 저장
            arrow_filename = f"{dataset_name}.arrow"
            dataset_obj.save_to_disk(arrow_filename)
            print(f"Saved {arrow_filename}")

print("All files have been saved.")

In [ ]:
import os
import shutil
import pandas as pd

# 디렉토리 경로 설정
base_dir = r""
newline_dir = os.path.join(base_dir, "newline")
mkdown_dir = os.path.join(base_dir, "mkdown")
newlinemkdown_dir = os.path.join(base_dir, "newlinemkdown")

# 디렉토리가 없으면 생성
for dir_path in [newline_dir, mkdown_dir, newlinemkdown_dir]:
    os.makedirs(dir_path, exist_ok=True)

# 파일 이동 함수
def move_file(file_name, source_dir, dest_dir):
    source_path = os.path.join(source_dir, file_name)
    dest_path = os.path.join(dest_dir, file_name)
    if os.path.exists(source_path):
        shutil.move(source_path, dest_path)
        print(f"Moved {file_name} to {dest_dir}")
    else:
        print(f"{file_name} not found in {source_dir}")

# 파일 이동
datasets = ['validation', 'train_df', 'wiki']

for dataset in datasets:
    # newline 파일 이동
    move_file(f"{dataset}_newline.csv", base_dir, newline_dir)
    move_file(f"{dataset}_newline.arrow", base_dir, newline_dir)
    
    # mkdown 파일 이동
    move_file(f"{dataset}_mkdown.csv", base_dir, mkdown_dir)
    move_file(f"{dataset}_mkdown.arrow", base_dir, mkdown_dir)
    
    # newlinemkdown 파일 이동
    move_file(f"{dataset}.csv", base_dir, newlinemkdown_dir)
    move_file(f"{dataset}.arrow", base_dir, newlinemkdown_dir)

print("File moving completed.")

In [ ]:
#test~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [379]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# 먼저 test가 pandas DataFrame인지 확인합니다.
if not isinstance(test, pd.DataFrame):
    test = pd.DataFrame(test)

# CSV 파일로 저장
csv_filename = 'test.csv'
test.to_csv(csv_filename, index=False)
print(f"CSV 파일이 {csv_filename}로 저장되었습니다.")

# Arrow 파일로 저장
arrow_filename = 'test.arrow'
table = pa.Table.from_pandas(test)
with pa.OSFile(arrow_filename, 'wb') as sink:
    with pa.RecordBatchFileWriter(sink, table.schema) as writer:
        writer.write_table(table)
print(f"Arrow 파일이 {arrow_filename}로 저장되었습니다.")




CSV 파일이 test.csv로 저장되었습니다.
Arrow 파일이 test.arrow로 저장되었습니다.


In [226]:
tarin1 = Dataset.from_pandas(train_df)
tarin1

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 3952
})

In [224]:
test1 = Dataset.from_pandas(test)
test1

Dataset({
    features: ['question', 'id'],
    num_rows: 600
})

In [ ]:
#사용할 함수들
#wiki_neline 
#wiki_mkdown
#wiki

In [32]:
wiki.head()

,text,corpus_source,url,domain,title,author,html,document_id
0,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",위키피디아,TODO,None,나라 목록,None,None,0
1,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,위키피디아,TODO,None,나라 목록,None,None,1
2,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,위키피디아,TODO,None,백남준,None,None,2
3,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",위키피디아,TODO,None,아오조라 문고,None,None,3
4,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,위키피디아,TODO,None,아오조라 문고,None,None,4


In [ ]:
import csv
import json

def csv_to_json(csv_file_path, json_file_path):
    # CSV 파일 읽기
    with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        data = list(csv_reader)

    # JSON 파일로 쓰기
    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)

# 사용 예시
csv_file_path = '.csv'
json_file_path = 'wikimkdown.json'
csv_to_json(csv_file_path, json_file_path)

print("CSV 파일이 JSON 파일로 성공적으로 변환되었습니다.")

In [38]:
# wikipedia 데이터 불러오기
# with open("C:/Users/black/OneDrive/바탕 화면/aaaaaa/data/wikipedia_documents.json", "r", encoding="utf-8") as f:
#     wiki = json.load(f)
with open(".json", "r", encoding="utf-8") as f:
    wiki = json.load(f)



wiki = pd.DataFrame(wiki)
wiki = wiki.transpose() # 필요한 column만 남기기
wiki.head(10)

,0,1,2,3,4,5,6,7,8,9,...,60603,60604,60605,60606,60607,60608,60609,60610,60611,60612
text,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,아오조라 문고는 자원봉사로 운영되며 열람 역시 무료이다. 서비스 개시 초반에는 보이...,"텍스트 파일을 아오조라 문고에 수록할 때, 텍스트 파일이 갖추어야 할 서식을 '아오...",일본어 표기에 많이 쓰이는 올림문자 (후리가나)는 그대로 올려쓰지 않고 '｜'나 '...,켄 톰슨(Ken Thompson) 등과 함께 최초의 유닉스(Unix) 시스템을 개발...,"주기율표(週期律表, 주기률표, periodic table) 또는 주기표(週期表)는 ...",...,"재래식 전력과 비대칭 전력, 즉 핵까지 갖고 있다한 들, 독자적으로 GPS(GNSS...","소련은 1960년 10월, 알제리를 독립 정부로 최초로 인정해준 나라였다. 소련이 ...","미국 안보 매체, 내셔널 인터레스트지는 한국군이 국방비를 늘렸다고, 비스트 모드(짐...","곰 자체는 페름 지역 숲에 사는 동물이 풍부함을 상징하며, 은색인 이유는 금속, 소...","1967년 소련 통치기 페름 시 의회는 새 문장 공모전을 개최하였으며, 1968년까...","오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...","1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,이 협약은 부당노동행위 제도를 규율하고 있다. 협약 제1조에서 반노동조합 차별행위로...
corpus_source,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,...,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아,위키피디아
url,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,TODO,...,,,,,,,,,,
domain,,,,,,,,,,,...,,,,,,,,,,
title,나라 목록,나라 목록,백남준,아오조라 문고,아오조라 문고,아오조라 문고,아오조라 문고,아오조라 문고,데니스 리치,주기율표,...,자주국방론,자주국방론,자주국방론,페름의 문장,페름의 문장,오키나와 현립 박물관·미술관,오키나와 현립 박물관·미술관,오키나와 현립 박물관·미술관,단결권 및 단체교섭권 협약,단결권 및 단체교섭권 협약
author,,,,,,,,,,,...,,,,,,,,,,
html,,,,,,,,,,,...,,,,,,,,,,
document_id,0,1,2,3,4,5,6,7,8,9,...,60603,60604,60605,60606,60607,60608,60609,60610,60611,60612
